# 5 AutoGen - Enabling LLM-powered agents to cooperate

The AutoGen ecosystem provides everything you need to create AI agents, especially multi-agent workflows -- framework, developer tools, and applications.

The framework uses a layered and extensible design. Layers have clearly divided responsibilities and build on top of layers below. This design enables you to use the framework at different levels of abstraction, from high-level APIs to low-level components.

Core API implements message passing, event-driven agents, and local and distributed runtime for flexibility and power. It also support cross-language support for .NET and Python.
AgentChat API implements a simpler but opinionated API for rapid prototyping. This API is built on top of the Core API and is closest to what users of v0.2 are familiar with and supports common multi-agent patterns such as two-agent chat or group chats.
Extensions API enables first- and third-party extensions continuously expanding framework capabilities. It support specific implementation of LLM clients (e.g., OpenAI, AzureOpenAI), and capabilities such as code execution.

<img src="https://github.com/microsoft/autogen/raw/main/autogen-landing.jpg" width=400>

> https://microsoft.github.io/autogen/stable/<br>
> https://github.com/microsoft/autogen

In [1]:
import autogen
import datetime
import openai
import os
import sys

from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from getpass import getpass
from IPython.display import Markdown, display
from PIL import Image

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 20-May-2025 14:28:09


In [5]:
WORK_DIR = "workdir"

os.makedirs(WORK_DIR, exist_ok=True)

## Multi agents cooperation examples

### 1. Financial analysis

In [6]:
config_list = config_list_from_json(env_or_file="CONFIG_LIST")

In [7]:
llm_config = {
    "cache_seed": None,
    "temperature": 0.,
    "config_list": config_list,
}

In [8]:
financial_tasks = [
    """What are the current stock prices of SALESFORCE and GOOGLE, and how is the performance over the past 
    month in terms of percentage change?""",
    """Investigate possible reasons of the stock performance leveraging market news.""",
]

writing_tasks = [
    """Develop an engaging blog post using any information provided."""
]

In [9]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)

research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)

writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

user_proxy_auto = autogen.UserProxyAgent(
    name="User_Proxy_Auto",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").
    rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": WORK_DIR,
        "use_docker": False,
    },  
)

user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="ALWAYS",  # ask human for input at each step
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").
    rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": WORK_DIR,
        "use_docker": False,
    }, 
)

chat_results = autogen.initiate_chats([
    {
        "sender": user_proxy_auto,
        "recipient": financial_assistant,
        "message": financial_tasks[0],
        "clear_history": True,
        "silent": False,
        "summary_method": "last_msg",
    },
    {
        "sender": user_proxy_auto,
        "recipient": research_assistant,
        "message": financial_tasks[1],
        "max_turns": 2,  
        "summary_method": "reflection_with_llm",
    },
    {
        "sender":
        user_proxy,
        "recipient":
        writer,
        "message":
        writing_tasks[0],
        "carryover":
        "I want to include a figure or a table of data in the blogpost.",  
    },
])


********************************************************************************
Starting a new chat....

********************************************************************************
User_Proxy_Auto (to Financial_assistant):

What are the current stock prices of SALESFORCE and GOOGLE, and how is the performance over the past 
    month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_assistant (to User_Proxy_Auto):

To gather the current stock prices of Salesforce (CRM) and Google (GOOGL), as well as their performance over the past month in terms of percentage change, I will use Python to fetch the required data from a financial API.

Here’s the plan:
1. Use the `yfinance` library to get the current stock prices and historical data for the past month.
2. Calculate the percentage change for each stock over the past month.

First, I will provide you with the code to install the necessary library and fetch the 

Replying as User_Proxy. Provide feedback to writer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (fd94e419-7329-4385-b71b-8c8024ddfc2d): User requested to end the conversation


In [11]:
for i, chat_res in enumerate(chat_results):
    print("-" * 100)
    print(f"{i+1}th chat:\n")
    print(chat_res.summary)
    print("\n1. Human input in the middle:", chat_res.human_input)
    print("\n2. Conversation cost: ", chat_res.cost)
    print()

----------------------------------------------------------------------------------------------------
1th chat:

The stock data has been successfully retrieved! Here are the results:

- **Salesforce (CRM)**: 
  - Current Price: $287.63
  - Percentage Change over the past month: 1688.74%

- **Google (GOOGL)**: 
  - Current Price: $166.54
  - Percentage Change over the past month: 62.16%

These values indicate significant increases in stock prices for both companies over the past month. 

If you need any further assistance or additional information, feel free to ask. 



1. Human input in the middle: []

2. Conversation cost:  {'usage_including_cached_inference': {'total_cost': 0.0032776499999999996, 'gpt-4o-mini-2024-07-18': {'cost': 0.0032776499999999996, 'prompt_tokens': 12199, 'completion_tokens': 2413, 'total_tokens': 14612}}, 'usage_excluding_cached_inference': {'total_cost': 0.0032776499999999996, 'gpt-4o-mini-2024-07-18': {'cost': 0.0032776499999999996, 'prompt_tokens': 12199, 'co

### Writing tasks

In [12]:
def my_writing_task(sender, recipient, context):
    carryover = context.get("carryover", "")
    if isinstance(carryover, list):
        carryover = carryover[-1]

    try:
        filename = context.get("work_dir", "") + "/stock_prices.md"
        with open(filename, "r") as file:
            data = file.read()
    except Exception as e:
        data = f"An error occurred while reading the file: {e}"

    return (
        """Develop an engaging blog post using any information provided. """ +
        "\nContext:\n" + carryover + "\nData:" + data)

In [13]:
research_task = "What are daily stock prices of NVDA and TESLA in the past month. Save the results in a .md file named 'stock_prices.md'."

In [14]:
researcher = autogen.AssistantAgent(
    name="Financial_researcher",
    llm_config=llm_config,
)

writer = autogen.AssistantAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

user_proxy_auto = autogen.UserProxyAgent(
    name="User_Proxy_Auto",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").
    rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": WORK_DIR,
        "use_docker": False,
    },
)

chat_results = autogen.initiate_chats([
    {
        "sender": user_proxy_auto,
        "recipient": researcher,
        "message": research_task,
        "clear_history": True,
        "silent": False,
        "summary_method": "last_msg",
    },
    {
        "sender": user_proxy_auto,
        "recipient": writer,
        "message": my_writing_task,
        "max_turns": 2,
        "summary_method": "reflection_with_llm",
        "work_dir": WORK_DIR,
    },
])


********************************************************************************
Starting a new chat....

********************************************************************************
User_Proxy_Auto (to Financial_researcher):

What are daily stock prices of NVDA and TESLA in the past month. Save the results in a .md file named 'stock_prices.md'.

--------------------------------------------------------------------------------
Financial_researcher (to User_Proxy_Auto):

To collect the daily stock prices of NVIDIA (NVDA) and Tesla (TSLA) for the past month, I will use the `yfinance` library in Python. The plan is as follows:

1. Use the `yfinance` library to fetch the daily stock prices for NVDA and TSLA for the past month.
2. Save the results in a Markdown file named `stock_prices.md`.

First, I will provide the code to install the necessary library and then fetch the stock prices.

```python
# filename: stock_prices.py
import yfinance as yf
from datetime import datetime, timedelta

In [15]:
!ls $WORK_DIR/stock_prices.md -lh

-rwxrwxrwx 1 root root 2.3K May 20 14:32 workdir/stock_prices.md


In [16]:
with open(os.path.join(WORK_DIR, "stock_prices.md"), 'r') as file:
    markdowncontent = file.read()

display(Markdown(markdowncontent))

# Daily Stock Prices for NVDA and TSLA

## NVDA
| Date                |   ('Close', 'NVDA') |
|:--------------------|--------------------:|
| 2025-04-21 00:00:00 |               96.91 |
| 2025-04-22 00:00:00 |               98.89 |
| 2025-04-23 00:00:00 |              102.71 |
| 2025-04-24 00:00:00 |              106.43 |
| 2025-04-25 00:00:00 |              111.01 |
| 2025-04-28 00:00:00 |              108.73 |
| 2025-04-29 00:00:00 |              109.02 |
| 2025-04-30 00:00:00 |              108.92 |
| 2025-05-01 00:00:00 |              111.61 |
| 2025-05-02 00:00:00 |              114.5  |
| 2025-05-05 00:00:00 |              113.82 |
| 2025-05-06 00:00:00 |              113.54 |
| 2025-05-07 00:00:00 |              117.06 |
| 2025-05-08 00:00:00 |              117.37 |
| 2025-05-09 00:00:00 |              116.65 |
| 2025-05-12 00:00:00 |              123    |
| 2025-05-13 00:00:00 |              129.93 |
| 2025-05-14 00:00:00 |              135.34 |
| 2025-05-15 00:00:00 |              134.83 |
| 2025-05-16 00:00:00 |              135.4  |
| 2025-05-19 00:00:00 |              135.57 |
| 2025-05-20 00:00:00 |              132.93 |

## TSLA
| Date                |   ('Close', 'TSLA') |
|:--------------------|--------------------:|
| 2025-04-21 00:00:00 |             227.5   |
| 2025-04-22 00:00:00 |             237.97  |
| 2025-04-23 00:00:00 |             250.74  |
| 2025-04-24 00:00:00 |             259.51  |
| 2025-04-25 00:00:00 |             284.95  |
| 2025-04-28 00:00:00 |             285.88  |
| 2025-04-29 00:00:00 |             292.03  |
| 2025-04-30 00:00:00 |             282.16  |
| 2025-05-01 00:00:00 |             280.52  |
| 2025-05-02 00:00:00 |             287.21  |
| 2025-05-05 00:00:00 |             280.26  |
| 2025-05-06 00:00:00 |             275.35  |
| 2025-05-07 00:00:00 |             276.22  |
| 2025-05-08 00:00:00 |             284.82  |
| 2025-05-09 00:00:00 |             298.26  |
| 2025-05-12 00:00:00 |             318.38  |
| 2025-05-13 00:00:00 |             334.07  |
| 2025-05-14 00:00:00 |             347.68  |
| 2025-05-15 00:00:00 |             342.82  |
| 2025-05-16 00:00:00 |             349.98  |
| 2025-05-19 00:00:00 |             342.09  |
| 2025-05-20 00:00:00 |             344.712 |



In [17]:
plain_text = markdowncontent.replace("**", "").replace("[", "").replace("]", "")
print(plain_text)

# Daily Stock Prices for NVDA and TSLA

## NVDA
| Date                |   ('Close', 'NVDA') |
|:--------------------|--------------------:|
| 2025-04-21 00:00:00 |               96.91 |
| 2025-04-22 00:00:00 |               98.89 |
| 2025-04-23 00:00:00 |              102.71 |
| 2025-04-24 00:00:00 |              106.43 |
| 2025-04-25 00:00:00 |              111.01 |
| 2025-04-28 00:00:00 |              108.73 |
| 2025-04-29 00:00:00 |              109.02 |
| 2025-04-30 00:00:00 |              108.92 |
| 2025-05-01 00:00:00 |              111.61 |
| 2025-05-02 00:00:00 |              114.5  |
| 2025-05-05 00:00:00 |              113.82 |
| 2025-05-06 00:00:00 |              113.54 |
| 2025-05-07 00:00:00 |              117.06 |
| 2025-05-08 00:00:00 |              117.37 |
| 2025-05-09 00:00:00 |              116.65 |
| 2025-05-12 00:00:00 |              123    |
| 2025-05-13 00:00:00 |              129.93 |
| 2025-05-14 00:00:00 |              135.34 |
| 2025-05-15 00:00:00 |         